In [5]:
# get the required table from wikipedia
import pandas as pd
import requests

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

wiki_df = pd.read_html(website_url,skiprows=0)[0]

wiki_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
# remove 'Not assigned' Boroughs
wiki_df = wiki_df[wiki_df['Borough'] != 'Not assigned']

# check the known issue (some Neighbourhoods could be 'Not assigned')
print(wiki_df[wiki_df['Neighbourhood'] == 'Not assigned'])


  Postcode       Borough Neighbourhood
8      M7A  Queen's Park  Not assigned


In [7]:
#fix the rows that have 'Not assigned' Neighbourhoods so that the Neighbourhood shows the Borough for these cases
for i, row in wiki_df.iterrows():
    if (row[2] == "Not assigned"):
        row[2] = row[1]

# check to see if M7A is 'fixed'
print(wiki_df[wiki_df['Postcode'] == 'M7A'])

  Postcode       Borough Neighbourhood
8      M7A  Queen's Park  Queen's Park


In [8]:
# group the Neighbourhoods by Postcode and Borough
wiki_df = wiki_df.groupby(['Postcode','Borough']).Neighbourhood.unique().apply(lambda x: ', '.join(x)).reset_index()

# check to see if the grouping worked
wiki_df[wiki_df['Postcode'] == 'M5A']

,Postcode,Borough,Neighbourhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [9]:
wiki_df.shape

# 103 records with three columns

(103, 3)